In [11]:
import cv2
import numpy as np
import os
from random import shuffle
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import tflearn
from sklearn.model_selection import train_test_split
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix


In [12]:
TRAIN_DIR = 'Train'
IMG_SIZE = 224
LR = 0.001
MODEL_NAME = 'Sports-Image'

In [13]:
def create_label(image_name):
    """ Create one-hot encoded vector from image name """
    word_label = image_name.split('_')[-2]
    if word_label == 'Basketball':
        return np.array([1,0,0,0,0,0])
    elif word_label == 'Football':
        return np.array([0,1,0,0,0,0])
    elif word_label == 'Rowing':
        return np.array([0,0,1,0,0,0])
    elif word_label == 'Swimming':
        return np.array([0,0,0,1,0,0])
    elif word_label == 'Tennis':
        return np.array([0,0,0,0,1,0])
    elif word_label == 'Yoga':
        return np.array([0,0,0,0,0,1])
    else:
        return None


In [14]:
images=[]
labels=[]
for i in os.listdir('C:/Users/Hp/Documents/fourth/first/DEEPLR/NNProj/jub/Train/'):

    img = cv2.imread('C:/Users/Hp/Documents/fourth/first/DEEPLR/NNProj/jub/Train/'+i)
    if img.all==None:
        continue
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    label=create_label(i)
    if label.all ==None:
        continue

    labels.append(label)
    images.append(img)
x = np.array(images)
y = np.array(labels)


In [15]:
x_train, x_val, y_train, y_val = train_test_split(x, y,
    test_size=0.2, shuffle = True, random_state = 8)

#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
 #   test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


##normlization
x_train = x_train/255

x_val=x_val/255


In [16]:
ops.reset_default_graph()
conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')
conv1 = conv_2d(conv_input, 32, 5, activation='relu')
pool1 = max_pool_2d(conv1, 5)

conv2 = conv_2d(pool1, 64, 5, activation='relu')
pool2 = max_pool_2d(conv2, 5)

conv3 = conv_2d(pool2, 128, 5, activation='relu')
pool3 = max_pool_2d(conv3, 5)

conv4 = conv_2d(pool3, 64, 5, activation='relu')
pool4 = max_pool_2d(conv4, 5)

conv5 = conv_2d(pool4, 32, 5, activation='relu')
pool5 = max_pool_2d(conv5, 5)
fully_layer = fully_connected(pool5, 1024, activation='relu')
fully_layer = dropout(fully_layer, 0.5)

cnn_layers = fully_connected(fully_layer, 6, activation='softmax')

cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)

In [8]:
if (os.path.exists('model.tfl.meta')):
    model.load('./model.tfl')
else:
    model.fit({'input': x_train}, {'targets': y_train}, n_epoch=100,
          validation_set=({'input': x_val}, {'targets': y_val}),
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    model.save('model.tfl')




INFO:tensorflow:Restoring parameters from C:\Users\Hp\Documents\fourth\first\DEEPLR\NNProj\jub\model.tfl


In [19]:
test_data = []

for path in os.listdir('C:/Users/Hp/Documents/fourth/first/DEEPLR/NNProj/jub/Test/'):
    
    
    img = cv2.imread('C:/Users/Hp/Documents/fourth/first/DEEPLR/NNProj/jub/Test/'+path)
    if img.all==None:
        
        
        continue
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    #img = img.reshape(244, 244, 1)
    prediction = model.predict([img])[0]
    
    max =0
    x = 0
    index = 0
    for i in  range(len(prediction)):
        if prediction[i] >max:
            max=prediction[i]
            index=x
            
                
        x+=1
        
        
    test_data.append([path,index])

        

import pandas as pd
df = pd.DataFrame(data=test_data,columns=['image_name','label'])

df.to_csv('Test.csv',index=False)
    

In [20]:
   print(len(test_data))

688
